In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Chicken-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size:int
    

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self)-> EvaluationConfig:
        evaluation_config=EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE    
        )

        return evaluation_config

In [7]:
import tensorflow as tf

In [10]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config

    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score= self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores={"loss": self.score[0], "accuracy":self.score[1]}
        save_json("scores.json", scores)
    

In [11]:
try:
    config=ConfigurationManager()
    val_config=config.get_evaluation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-12-18 13:25:11,802: INFO: common: Created dirctory at: artifacts]
[2025-12-18 13:25:11,986: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 116 images belonging to 2 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6034 - loss: 2.2196
